In [1]:
import fn_file as fn

import itertools
from gensim.models import Word2Vec
import pandas as pd
import copy
import re

from khaiii import KhaiiiApi
api = KhaiiiApi()
from multiprocessing import Pool
import time

In [2]:
def clean_str(text):
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '<[^>]*>'         # HTML 태그 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '[^\w\s]'         # 특수기호제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]' # 특수기호 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = "\u3000\u3000"     # 유니코드 삭제
    text = re.sub(pattern=pattern, repl='',string=text)
    pattern = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE) #마지막점검 제거
    text = re.sub(pattern=pattern, repl='',string=text)
    return text

## 0. 데이터와 w2v 모델 불러오기

In [3]:
morphs =[[]]
with open('for_word2vec.txt') as f:
    for i in range(0,115071):
        for line in f:
            morphs[i].append(line.split(','))
morphs=morphs[0]

for i in range(0,len(morphs)):
    morphs[i][-1] = morphs[i][-1].rstrip('\n')

In [4]:
# w2v모델 불러오기
model_tag = Word2Vec.load('play_w2v.model')

In [5]:
train_data = pd.read_json('train.json', encoding='UTF-8', typ = 'frame')
valid_data = pd.read_json('val.json', encoding='UTF-8', typ = 'frame')
song_meta = pd.read_json('song_meta.json', encoding='UTF-8', typ = 'frame')
genre_gn_all = pd.read_json('genre_gn_all.json',encoding='UTF-8', typ = 'series')
genre_gn_all = pd.DataFrame(genre_gn_all, columns = ['gnr_name']).reset_index().rename(columns = {'index' : 'gnr_code'})

for_genre = copy.deepcopy(song_meta[['song_gn_dtl_gnr_basket','id']])

In [6]:
uniq_tag = list(set(itertools.chain(*train_data.tags)))

for i in range(0,len(uniq_tag)):
    uniq_tag[i] = uniq_tag[i].lower()

In [7]:
# validation data 복사본 만들어두기
prac = copy.deepcopy(valid_data[['tags','plylst_title','songs','updt_date','like_cnt']])

In [8]:
for i in range(0,len(prac)):
    mylist = [[prac['plylst_title'][i]]]
    clean = clean_str(' '.join(sum(mylist,[])))
    if len(clean)==0:
        clean=' '
    prac['plylst_title'][i] = clean
    
for i in range(len(prac['plylst_title'])):
    prac['plylst_title'][i]=prac['plylst_title'][i].strip()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


## 1. 형태소 추출

In [9]:
def do_work(i):
    tmp_list = []
    noun_list = []
    if len(prac['plylst_title'][i]) == prac['plylst_title'][i].count(' '):
        # 빈칸으로만 되어있는 행
        pass
    elif prac['plylst_title'][i].find(" ")==-1:
        # 띄어쓰기 없는 행
        tmp_list.append(prac['plylst_title'][i])
        noun_list.append(prac['plylst_title'][i])
    else:
        for word in api.analyze(prac['plylst_title'][i]):
            for morph in word.morphs:
                # 실질형태소 가져오기
                if morph.tag in ['NNG', 'NNP', 'NP', 'VV', 'VA', 'XR', 'IC', 'MM', 'MAG', 'MAJ','SN','SL']:
                    tmp_list.append(morph.lex)
                if morph.tag in ['NNG','NNP','NP','SL']:
                    noun_list.append(morph.lex)
    if i%100==0:
        print(i)
    if i%1000==0:
        time.sleep(5)
        
    return [tmp_list, noun_list]



if __name__ == '__main__':
    start_time = time.time()
    with Pool(processes=7) as p:
        result= p.map(do_work, range(0,len(prac['plylst_title'])))
    print("--- %s seconds ---" % (time.time()-start_time))

03300900

25001700
5000


4200
3400
100018002600


4300
3500
44001900
2700

45003600

2800
2000
3700
4600
2900
3800
4700
3000
39004800

4900
4000
5800
5900
6000
100
5100
200
5200
300
400
5300
5001100

5400
1200
5500600

1300
5600
700
1400
5700
800
7400
6600
1500
2100
7500
6700
1600
2200
68007600

8300
23007700

69003100

7800
8400
70002400

79003200

41008500

1070080009100
9900


8600
108009200

10000
8700
6100
10900
88009300

6200
110009400

6300
8900
9500
90006400

9600
6500
9700
11600
9800
11700
12400
12500
11800
12600
11900
1200012700

12800
12900
13000
7100
7200
7300
8100
13200
8200
13300
1400010100

10200
13400
1350010300

1040013600

11100
10500
13700
11200
1480010600
13800

11300
1390014900

11400
15700
15000
1650015800

11500
1730015900

16600
17400
16000
16700
12100
17500
16800
17600
12200
1690017700

12300
1310018100

1700017800

1820017900

19000
1800018300

18400
18500
18600
18700
18800
18900
19800
19900
20000
14100
14200
14300
14400
14500
14600
14700
20600
20700
20800
15

### 1-1. 형태소 추출 된것 전처리

In [10]:
### 영어 소문자로
for i in range(len(result)):
    for j in range(len(result[i])):
        for z in range(len(result[i][j])):
            result[i][j][z] = result[i][j][z].lower()

### 영어 한글자 지우기
import re
p = re.compile('[a-z]')

for i in range(len(result)):
    for j in range(len(result[i])):
        for z in range(len(result[i][j])):
            if len(result[i][j][z]) == 1:
                if p.match(result[i][j][z]) != None:
                    result[i][j][z] = ''

for i in range(len(result)):
    for j in range(len(result[i])):
        while '' in result[i][j]:
            result[i][j].remove('')
            
### 중복되는 단어 제거
for i in range(len(result)):
    for j in range(len(result[i])):
        result[i][j] = list(set(result[i][j]))

### 숫자 4자리 제외 제거
for i in range(len(result)):
    for j in range(len(result[i])):
        for z in range(len(result[i][j])):
            if result[i][j][z].isdigit() == True:
                if len(result[i][j][z]) != 4:
                    result[i][j][z] = ''
                    
for i in range(len(result)):
    for j in range(len(result[i])):
        while '' in result[i][j]:
            result[i][j].remove('')

In [11]:
full_list = []
noun_list = []

for lst in result:
    full_list.append(lst[0])
    noun_list.append(lst[1])

### 1-2. validation 데이터 완성(prac)
 - 미연의 일을 방지하기 위해서 valid의 복사본인 prac 만듦.

In [12]:
prac['plylst_title']=copy.deepcopy(full_list)
prac['nouns']=copy.deepcopy(noun_list)
pred_tag=[]
pred_song=[]
for i in range(len(prac)):
    pred_tag.append([])
for i in range(len(prac)):
    pred_song.append([])
prac['pred_song']=pred_song
prac['pred_tag']=pred_tag
prac['index'] = range(len(prac))
prac = prac[['index','tags','plylst_title','nouns','songs','updt_date','pred_tag','pred_song','like_cnt']]

### 들어있는 tag 영어 소문자로
for i in range(0,len(prac['tags'])):
    for j in range(0, len(prac['tags'][i])):
        prac['tags'][i][j] = prac['tags'][i][j].lower()

every_thing=[]
for i in range(0,len(prac)):
    every_thing.append(prac.iloc[i,:].tolist())
cop_every = copy.deepcopy(every_thing)

## 2. 예측 Modeling

In [14]:
# 최종 Modeling

for lst in cop_every[1:30]:
    j=0
    if len(lst[1]) == 0 and len(lst[2]) !=0 and len(lst[4])==0:
        fn.fn_only_play(lst,model_tag,uniq_tag,30,morphs,train_data)
    elif len(lst[1]) != 0 and len(lst[2]) !=0 and len(lst[4])==0:
        fn.fn_tag_play(lst,model_tag,uniq_tag,30,morphs,train_data)
    elif len(lst[1]) != 0 and len(lst[2]) ==0 and len(lst[4])==0:
        fn.fn_only_tag(lst,model_tag,uniq_tag,30,morphs,train_data)
    elif len(lst[1]) != 0 and len(lst[2]) ==0 and len(lst[4])!=0:
        fn.fn_tag_song(lst,model_tag,uniq_tag,30,morphs,train_data)
    elif len(lst[1]) == 0 and len(lst[2]) ==0 and len(lst[4])!=0:
        fn.fn_only_song(lst,model_tag,for_genre,uniq_tag,genre_gn_all,30,train_data)

No vocab
#------- Finished  2  th of  30  data -------#
#------- Finished  3  th of  30  data -------#
#------- Finished  4  th of  30  data -------#
#------ Finished  5  th of  30  --------#
#------- Finished  6  th of  30  data -------#
#------- Finished  7  th of  30  data -------#
#--------- doing 8th of 30 ------------#
#----------- doing unlist -------------#
#--------- Finished 8th took 0.016666666666666666mins ------------#
#--------- doing 9th of 30 ------------#
#----------- doing unlist -------------#
#--------- Finished 9th took 0.03333333333333333mins ------------#
#------- Finished  10  th of  30  data -------#
#------ Finished  11  th of  30  --------#
#------ Finished  12  th of  30  --------#
#------- Finished  13  th of  30  data -------#
#------- Finished  14  th of  30  data -------#
#------- Finished  15  th of  30  data -------#
#------- Finished  16  th of  30  data -------#
#--------- doing 17th of 30 ------------#
#----------- doing unlist -------------#
#-----

### 2-1. 예측 된 것 확인

In [17]:
pd.DataFrame(cop_every).iloc[:30,:]

,0,1,2,3,4,5,6,7,8
0,0,[],[],[],"[373313, 151080, 275346, 696876, 165237, 52593...",2019-05-27 14:14:33.000,[],[],1675
1,1,[],[앨리스테이블],[앨리스테이블],[],2014-07-16 15:24:24.000,[앨리스테이블],[],1
2,2,[],[],[],"[529437, 516103, 360067, 705713, 226062, 37089...",2008-06-21 23:26:22.000,"[모음, 솔로, 오디오, 플레이리스트, 태연, bgm, 2020, 상시, 2010년...",[],62
3,3,[],[],[],"[589668, 21711, 570151, 320043, 13930, 599327,...",2017-10-30 18:15:43.000,"[포크, 밴드, 뮤지션, 싱어송라이터, 인디, 얼터너티브, 아날로그, 2000, 한...",[],20
4,4,[],[],[],"[672718, 121924, 102694, 683657, 201558, 38511...",2017-02-07 11:40:42.000,"[ccm, 워십, 찬양, 은혜, 국내ccm, 인기, 추천, 좋아하, 교회, 아이, ...",[],20
5,5,"[비, 분위기, 컨트리, 서정적인]",[],[],"[173033, 138237, 63140, 15733, 461780, 308519,...",2019-06-12 17:43:35.000,"[jazz, 포크, 블루스, 거리, 어쿠스틱, 포크/블루스/컨트리, 인디, 올드팝,...",[],2
6,6,[],[],[],"[227044, 424672, 515592, 22408, 57893, 153714,...",2011-10-28 11:21:51.000,"[일렉트로닉, 하우스, 신스팝, party, club, 트랜스, 테크하우스, mus...",[],2
7,7,[],[],[],"[54457, 423722, 337682, 91214, 644980, 556444,...",2014-01-26 20:00:30.000,"[일렉, dance, 하우스, house, 디스코, 테크노, 파티, 트랜스, 일렉트...",[],3
8,8,[스트레스],"[리, 운동, 스트레스, 듬타, 빡시, 날]","[스트레스, 운동, 리, 날]",[],2017-02-09 17:33:45.000,"[기분, 신나, 흔들, up, 전환, 운동, 달리, 여름, 성인가요, 트렌디]","[459256, 355939, 254865, 130199, 445170, 27703...",127
9,9,[],"[하루, 좋, 만들, 기분, 재즈, 달, 함께]","[기분, 재즈, 하루, 달]",[],2015-06-22 09:11:02.000,"[시작, 오늘, 연인, 하루, jazz, 기분, 재즈, 커피, 함께, 위로]","[663256, 177460, 705515, 140867, 366786, 32172...",0
